[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/monacofj/moeabench/blob/main/examples/example_05.ipynb)

# Example 05: Custom Benchmark Implementation

This example demonstrates how to extend MoeaBench by creating your 
own vectorized multi-objective problem (MOP), allowing you to optimize 
custom domains with the same analytical tools.

In [ ]:
# Install MoeaBench from GitHub
!pip install --quiet git+https://github.com/monacofj/moeabench.git


In [ ]:



from MoeaBench import mb
print(f"MoeaBench v{mb.system.version()}")
import numpy as np

class MyProblem(mb.mops.BaseMop):
    def __init__(self):
        # M=3 Objectives, N=10 Variables, Bounds=[0, 1]
        # Providing a 'name' enables Sidecar Diagnostic generation (v0.9+)
        super().__init__(name="MyCustomProblem_3D_v1", M=3, N=10, xl=0.0, xu=1.0)

    def evaluation(self, X, n_ieq_constr=0):
        # Evaluation MUST be vectorized (X is a population matrix)
        # Objectives are mapped to a spherical surface (DTLZ2-style)
        theta = X[:, :2] * (np.pi / 2)
        g = np.sum((X[:, 2:] - 0.5)**2, axis=1)
        
        f1 = (1 + g) * np.cos(theta[:, 0]) * np.cos(theta[:, 1])
        f2 = (1 + g) * np.cos(theta[:, 0]) * np.sin(theta[:, 1])
        f3 = (1 + g) * np.sin(theta[:, 0])
        
        # Return objectives in dictionary 'F' (pop_size x 3)
        return {'F': np.column_stack([f1, f2, f3])}

    def ps(self, n):


### Interpretation

Creating custom problems is as simple as defining the 'evaluation' function. 
Because MoeaBench expects a vectorized input (X), you can leverage 
the full speed of NumPy.

Once defined, your custom mops becomes a "first-class citizen" 
in the library, compatible with all plotting tools and statistical measures.